In [4]:
%cd /home/mrearle/repos/R2R-EnvDrop-ObjAttn/

/mnt-homes/kraken/mrearle/repos/R2R-EnvDrop-ObjAttn


# Imports

In [5]:
import sys
import torch

metadata_parser_path = '/home/mrearle/repos/R2R-EnvDrop-ObjAttn/metadata_parser'
if metadata_parser_path not in sys.path:
    sys.path.append(metadata_parser_path)

mattersim_path = '/home/mrearle/datasets/Matterport3DSimulator/build'
if mattersim_path not in sys.path:
    sys.path.append(mattersim_path)
    
src_path = '/home/mrearle/repos/R2R-EnvDrop-ObjAttn/r2r_src'
if src_path not in sys.path:
    sys.path.append(src_path)


In [6]:
from parse_house_segmentations import HouseSegmentationFile

# Parameters

In [7]:
from param import args

Optimizer: Using RMSProp
Using device: cuda
_CudaDeviceProperties(name='NVIDIA GeForce GTX 1080 Ti', major=6, minor=1, total_memory=11178MB, multi_processor_count=28)


	Training model default in experiment default




In [8]:
# Model args
args.name = 'agent_bt_val'
args.attn = 'soft'
args.train = 'validlistener'
args.angle_feat_size = 128
args.accumulateGrad = True
args.featdropout = 0.4
args.subout = 'max'
args.optim = 'rms'
args.lr = 1e-4
args.iters = 10
args.maxAction = 35

# Obj attn args
base = False
args.obj_attn_type = "connection"
args.max_obj_number = 32
args.obj_aux_task = not base
args.obj_aux_task_weight = 0.1
args.dataset = "R2R"
args.include_objs = not base
args.include_objs_lstm = False
args.reduced_envs = True
args.buffer_objs = False

# Setup Agent

## Imports

In [9]:
from agent import Seq2SeqAgent
from collections import defaultdict
import json
import numpy as np
import h5py
from collections import OrderedDict
from train import setup, read_vocab, Tokenizer, TRAIN_VOCAB, read_img_features, R2RBatch, Evaluation

log_dir = args.log_dir
FEATURE_FILE = "/home/mrearle/storage/img_features/ResNet-152-imagenet.hdf5"

Namespace(CANDIDATE_FEATURES='img_features/ResNet-152-candidate.tsv', IMAGENET_FEATURES='img_features/ResNet-152-imagenet.tsv', OBJECT_CLASS_FILE='/workspace1/mrearle/object_classes.json', OBJECT_FEATURES='/workspace1/mrearle/object_features_filtered.hdf5', TRAINVAL_VOCAB='tasks/R2R/data/trainval_vocab.txt', TRAIN_VOCAB='tasks/R2R/data/train_vocab.txt', accumulateGrad=True, accumulate_grad=False, aemb=64, alpha=0.5, angle_feat_size=128, attn='soft', aug=None, batchSize=64, beam=False, bidir=True, buffer_objs=False, candidate_mask=False, candidates=1, dataset='R2R', device=device(type='cuda'), dropout=0.5, encode='word', epsilon=0.1, experiment='default', fast_train=False, featdropout=0.4, feature_size=2048, features='imagenet', features_fast='img_features/ResNet-152-imagenet-fast.tsv', feedback='sample', gamma=0.9, ignoreid=-100, include_objs=True, include_objs_lstm=False, iters=10, listener=None, load=None, loadOptim=False, log_dir='snap/default/default', logging_vis=False, lr=0.0001,

## Environments

In [10]:
setup()

vocab = read_vocab(TRAIN_VOCAB)
tok = Tokenizer(vocab=vocab, encoding_length=args.maxInput)

feat_h5 = h5py.File(FEATURE_FILE, "r")
obj_feat_h5 = h5py.File(args.OBJECT_FEATURES, "r")

featurized_scans = set(feat_h5.keys())

train_env = R2RBatch(
    feature_store=feat_h5, object_feat_store=obj_feat_h5, batch_size=args.batchSize, splits=["train"], tokenizer=tok
)

val_env_names = ["val_unseen"]

val_envs = OrderedDict(
        (
            (
                split,
                (
                    R2RBatch(
                        feature_store=feat_h5,
                        object_feat_store=obj_feat_h5,
                        batch_size=args.batchSize,
                        splits=[split],
                        tokenizer=tok,
                    ),
                    Evaluation([split], featurized_scans, tok),
                ),
            )
            for split in val_env_names
        )
    )

W2I is defaultdict True
OLD_VOCAB_SIZE 991
VOCAB_SIZE 992
VOACB 991
The feature size is 2048
Loading navigation graphs for 21 scans
R2RBatch loaded with 4895 instructions, using splits: train
The feature size is 2048
Loading navigation graphs for 11 scans
R2RBatch loaded with 2349 instructions, using splits: val_unseen


## Setup Agent

In [8]:
agent = Seq2SeqAgent(train_env, "", tok, args.maxAction)

agent_name = 'agent-obj(32)-aux(0.1)-reduced'

args.load = f'snap/{agent_name}/state_dict/best_val_unseen'
agent.results_path = f'results/{agent_name}.json'
print(
    "Loaded the listener model at iter %d from %s"
    % (agent.load(args.load), args.load),
    flush=True,
)

Using Bidir in EncoderLSTM
ObjAttention: Using class ConnectionwiseObjectAttention
Listener: Done Instantiating Model. Initializing Optimizers
Listener: Done Instantiating Optimizers. Initializing Loss
Listener: Done Instantiating Loss. Initializing Logs
Listener: Flushed
Listener: Done Instantiating Logs. Listener Initialized
Loaded the listener model at iter 82500 from snap/agent-obj(32)-aux(0.1)-reduced/state_dict/best_val_unseen


In [9]:
for env_name, (env, evaluator) in val_envs.items():
    agent.logs = defaultdict(list)
    agent.env = env

    iters = None
    agent.test(use_dropout=False, feedback="argmax", iters=iters)
agent_result = agent.write_results()



KeyboardInterrupt: 

# Compare trajectory data

In [11]:
import json
import networkx as nx

In [12]:
ground_truth_path = 'tasks/R2R/data/R2R_val_unseen.json'
base_agent_path = 'results/agent_base-reduced.json'
obj_agent_path = 'results/agent-obj(32)-aux(0.1)-reduced.json'

In [22]:
def load_nav_graphs(scans):
    """Load connectivity graph for each scan"""

    def distance(pose1, pose2):
        """Euclidean distance between two graph poses"""
        return (
            (pose1["pose"][3] - pose2["pose"][3]) ** 2
            + (pose1["pose"][7] - pose2["pose"][7]) ** 2
            + (pose1["pose"][11] - pose2["pose"][11]) ** 2
        ) ** 0.5

    graphs = {}
    for scan in scans:
        with open("connectivity/%s_connectivity.json" % scan) as f:
            G = nx.Graph()
            positions = {}
            data = json.load(f)
            for i, item in enumerate(data):
                if item["included"]:
                    for j, conn in enumerate(item["unobstructed"]):
                        if conn and data[j]["included"]:
                            positions[item["image_id"]] = np.array([item["pose"][3], item["pose"][7], item["pose"][11]])
                            assert data[j]["unobstructed"][i], "Graph should be undirected"
                            G.add_edge(
                                item["image_id"],
                                data[j]["image_id"],
                                weight=distance(item, data[j]),
                            )
            nx.set_node_attributes(G, values=positions, name="position")
            graphs[scan] = G
    
    img_id_to_graph = {}
    for G in graphs.values():
        for img_id in list(G.nodes()):
            img_id_to_graph[img_id] = G
    return graphs, img_id_to_graph

In [23]:
graphs, img_id_to_graph = load_nav_graphs(featurized_scans)

In [69]:
def distance(vp1, vp2):
    """Euclidean distance between two graph poses"""
    pose1 = img_id_to_graph[vp1].nodes[vp1]["position"]
    pose2 = img_id_to_graph[vp2].nodes[vp2]["position"]
    return (
        (pose1[0] - pose2[0]) ** 2
        + (pose1[1] - pose2[1]) ** 2
        + (pose1[2] - pose2[2]) ** 2
    ) ** 0.5

def success(vp1, vp2):
    return distance(vp1, vp2) <= 3.0

In [46]:
with open(ground_truth_path) as f:
    ground_truth = json.load(f)
ground_truth = {v['path_id']: v['path'] for v in ground_truth}

with open(base_agent_path) as f:
    base_agent = json.load(f)
base_agent = {v['instr_id']: v['trajectory'] for v in base_agent}

with open(obj_agent_path) as f:
    obj_agent = json.load(f)
obj_agent = {v['instr_id']: v['trajectory'] for v in obj_agent}

In [64]:
succ_cases = []
fail_cases = []
both_succs = []
both_fails = []
for instr_id in base_agent:
    path_id, _ = instr_id.split('_')

    base_agent_success = success(base_agent[instr_id][-1][0], ground_truth[int(path_id)][-1])
    obj_agent_success = success(obj_agent[instr_id][-1][0], ground_truth[int(path_id)][-1])

    if not base_agent_success and obj_agent_success:
        succ_cases.append(instr_id)
    elif not obj_agent_success and base_agent_success:
        fail_cases.append(instr_id)
    elif base_agent_success and obj_agent_success:
        both_succs.append(instr_id)
    elif not base_agent_success and not obj_agent_success:
        both_fails.append(instr_id)

In [65]:
len(succ_cases), len(fail_cases), len(both_succs), len(both_fails)

(212, 235, 558, 1344)

In [53]:
with open('/home/mrearle/repos/compute_conv_feats/splitted_instructions_val_unseen.json', 'r') as f:
    obj_split_instruction = json.load(f)

In [70]:
many_obj = { 'success': [], 'fail': [] }
less_obj = { 'success': [], 'fail': [] }
many_obj_base = { 'success': [], 'fail': [] }
less_obj_base = { 'success': [], 'fail': [] }


for instr_id in base_agent:
    path_id, _ = instr_id.split('_')

    path_type = 'many_obj' if instr_id in obj_split_instruction['many_objs'] else 'less_obj'

    base_agent_success = success(base_agent[instr_id][-1][0], ground_truth[int(path_id)][-1])
    obj_agent_success = success(obj_agent[instr_id][-1][0], ground_truth[int(path_id)][-1])

    if path_type == 'many_obj':
        many_obj['success' if obj_agent_success else 'fail'].append(instr_id)
    else:
        less_obj['success' if obj_agent_success else 'fail'].append(instr_id)

    
    if path_type == 'many_obj':
        many_obj_base['success' if base_agent_success else 'fail'].append(instr_id)
    else:
        less_obj_base['success' if base_agent_success else 'fail'].append(instr_id)


In [71]:
print('Obj Agent')
print(len(many_obj['success']), len(many_obj['fail']), len(less_obj['success']), len(less_obj['fail']))
print('Many obj SR:', len(many_obj['success']) / (len(many_obj['success']) + len(many_obj['fail'])))
print('Less obj SR:', len(less_obj['success']) / (len(less_obj['success']) + len(less_obj['fail'])))

Obj Agent
389 950 381 629
Many obj SR: 0.2905153099327857
Less obj SR: 0.37722772277227723


In [72]:
print('Base Agent')
print(len(many_obj_base['success']), len(many_obj_base['fail']), len(less_obj_base['success']), len(less_obj_base['fail']))
print('Many obj SR:', len(many_obj_base['success']) / (len(many_obj_base['success']) + len(many_obj_base['fail'])))
print('Less obj SR:', len(less_obj_base['success']) / (len(less_obj_base['success']) + len(less_obj_base['fail'])))

Base Agent
408 931 385 625
Many obj SR: 0.30470500373412995
Less obj SR: 0.3811881188118812
